<div class="alert alert-block alert-info">A notebook for preprocessing the city metrics tables in preparation for the route opening analysis. The preprocessing involves a few data manipulations (e.g., reshaping from wide to long format, merging with scheduled data, applying filters, and adding a few KPIs such as logarithmic values or growth indicators).</div>

# Import

## lib

In [1]:
import numpy as np
import polars as pl
import pandas as pd
from polars import col as d
import glob
import os

## csv

In [2]:
folder_path = '/home/sara/Desktop/ATSLab/data/' 

In [3]:
df_cities_metrics_modif = pl.read_csv(folder_path+"df_cities_metrics_modif.csv")
df_scheduled = pl.read_parquet(folder_path+"scheduled_dataset_transatlantic_enhanced.parquet") ## change name
df_airports_lookup_modif = pl.read_csv(folder_path+"df_airports_lookup_modif.csv")

# Unpivot cities metrics table

In [4]:
df_cities_metrics_by_apt = (
    df_cities_metrics_modif
    .filter(d.REGION_ID.is_in([10,13]))
    .select(['METRO_ID',
    # 'METRO_CITY',
    # 'METRO_COUNTRY',
    # 'NB_APT_2015',

    'POPU_2010',
    'POPU_2011',
    'POPU_2012',
    'POPU_2013',
    'POPU_2014',
    'POPU_2015',

    'INC_2010_LC',
    'INC_2011_LC',
    'INC_2012_LC',
    'INC_2013_LC',
    'INC_2014_LC',
    'INC_2015_LC',
    'INC_2010_USD15',
    'INC_2011_USD15',
    'INC_2012_USD15',
    'INC_2013_USD15',
    'INC_2014_USD15',
    'INC_2015_USD15',

    'IS_SPECIAL',
    'IS_CAPITAL',
    'IS_GLOBAL_HUB',
    'IS_DOMESTIC_HUB',

    'APT_ID_1',
    'APT_ID_2',
    'APT_ID_3',
    'APT_ID_4',
    'APT_ID_5',
    'APT_ID_6',
    'APT_ID_7',
    'APT_ID_8',
    'APT_ID_9',
    'APT_ID_10',
    'APT_ID_11',
    'APT_ID_12',
    'APT_ID_13',

    ]
    )

    ## 2015 USD to 2019 USD
    .with_columns(
          INC_2010_USD19 = d.INC_2010_USD15 *1.08,
          INC_2011_USD19 = d.INC_2011_USD15 *1.08,
          INC_2012_USD19 = d.INC_2012_USD15 *1.08,
          INC_2013_USD19 = d.INC_2013_USD15 *1.08,
          INC_2014_USD19 = d.INC_2014_USD15 *1.08,
          INC_2015_USD19 = d.INC_2015_USD15 *1.08
    )

    .drop('INC_2010_USD15', 'INC_2011_USD15', 'INC_2012_USD15', 'INC_2013_USD15', 'INC_2014_USD15', 'INC_2015_USD15')


    ## wide to long format to have the indicators by airport and not by metropolitan area
    .unpivot(index = ['METRO_ID', 'POPU_2010', 'POPU_2011', 'POPU_2012', 'POPU_2013', 'POPU_2014', 'POPU_2015', 'INC_2010_LC',	'INC_2011_LC',	'INC_2012_LC',	'INC_2013_LC',	'INC_2014_LC',	'INC_2015_LC',	'IS_SPECIAL',	'IS_CAPITAL',	'IS_GLOBAL_HUB', 'IS_DOMESTIC_HUB', 'INC_2010_USD19', 'INC_2011_USD19', 'INC_2012_USD19', 'INC_2013_USD19', 'INC_2014_USD19', 'INC_2015_USD19'],
    on = ['APT_ID_1', 'APT_ID_2', 'APT_ID_3', 'APT_ID_4', 'APT_ID_5', 'APT_ID_6', 'APT_ID_7', 'APT_ID_8', 'APT_ID_9', 'APT_ID_10', 'APT_ID_11', 'APT_ID_12', 'APT_ID_13']
    )
    .rename({'variable':'WHICH_APT', 'value': 'APT_ID'})
    .filter(d.APT_ID != 0)

    .drop('WHICH_APT')

    ## add growth
    .with_columns(EVO_POPU_1011 = 100*(d.POPU_2011 - d.POPU_2010)/d.POPU_2010,
                  EVO_POPU_1112 = 100*(d.POPU_2012 - d.POPU_2011)/d.POPU_2011,
                  EVO_POPU_1213 = 100*(d.POPU_2013 - d.POPU_2012)/d.POPU_2012,
                  EVO_POPU_1314 = 100*(d.POPU_2014 - d.POPU_2013)/d.POPU_2013,
                  EVO_POPU_1415 = 100*(d.POPU_2015 - d.POPU_2014)/d.POPU_2014,

                  EVO_INC_LC_1011 = 100*(d.INC_2011_LC - d.INC_2010_LC)/d.INC_2010_LC,
                  EVO_INC_LC_1112 = 100*(d.INC_2012_LC - d.INC_2011_LC)/d.INC_2011_LC,
                  EVO_INC_LC_1213 = 100*(d.INC_2013_LC - d.INC_2012_LC)/d.INC_2012_LC,
                  EVO_INC_LC_1314 = 100*(d.INC_2014_LC - d.INC_2013_LC)/d.INC_2013_LC,
                  EVO_INC_LC_1415 = 100*(d.INC_2015_LC - d.INC_2014_LC)/d.INC_2014_LC,

                  EVO_INC_USD19_1011 = 100*(d.INC_2011_USD19 - d.INC_2010_USD19)/d.INC_2010_USD19,
                  EVO_INC_USD19_1112 = 100*(d.INC_2012_USD19 - d.INC_2011_USD19)/d.INC_2011_USD19,
                  EVO_INC_USD19_1213 = 100*(d.INC_2013_USD19 - d.INC_2012_USD19)/d.INC_2012_USD19,
                  EVO_INC_USD19_1314 = 100*(d.INC_2014_USD19 - d.INC_2013_USD19)/d.INC_2013_USD19,
                  EVO_INC_USD19_1415 = 100*(d.INC_2015_USD19 - d.INC_2014_USD19)/d.INC_2014_USD19,
                  
                #   EVO_POPU_1015 = 100*(d.POPU_2015 - d.POPU_2010)/d.POPU_2010,
              
                  )

    .drop('METRO_ID')
        
)

# Enhanced scheduled dataset

In [24]:
df_opening_apt_pair = (
    df_scheduled
    .filter(d.YEAR > 2009)
    .filter(d.YEAR < 2015)
    .filter(d.IS_OPENING)
    .select('APT_CODE_A', 'APT_CODE_B')
    .with_columns(NEW_OPENING = True)
)

In [33]:
df_prep_for_stat = (
    df_scheduled    

    .join(df_airports_lookup_modif.select(['APT_ID', 'APT_CODE']).rename({'APT_ID':'APT_ID_A', 'APT_CODE':'APT_CODE_A'}), how = 'left', on = 'APT_CODE_A') ## no null values that's good
    .join(df_airports_lookup_modif.select(['APT_ID', 'APT_CODE']).rename({'APT_ID':'APT_ID_B', 'APT_CODE':'APT_CODE_B'}), how = 'left', on = 'APT_CODE_B') ## same comment

    .filter(d.YEAR > 2009)
    .filter(d.YEAR < 2016)

    .join(df_cities_metrics_by_apt.rename({col: f"{col}_A" for col in df_cities_metrics_by_apt.columns}), how = 'left', on = 'APT_ID_A')
    .join(df_cities_metrics_by_apt.rename({col: f"{col}_B" for col in df_cities_metrics_by_apt.columns}), how = 'left', on = 'APT_ID_B')

    .filter(d.MKT_TYPE == 'INTER')

    ## overall
    ## 107_809 lines to 99_723 ## we loose 7.5% of our data
    ## concerning the opening
    # .filter(d.IS_OPENING) ## 5_957 to 5_037, we loose 15% of our data (not nice)

    ## for the international market
    ## 4_351 to 4_308 ## we loose less than 1% of our data
    ## concerning the opening 227 to 221 so it's ok
    .filter(~d.POPU_2010_A.is_null())
    .filter(~d.POPU_2010_B.is_null())


    .with_columns(POPU_A = pl.when(d.YEAR == 2010)
                             .then(d.POPU_2010_A)
                             .when(d.YEAR == 2011)
                             .then(d.POPU_2011_A)
                             .when(d.YEAR == 2012)
                             .then(d.POPU_2012_A)
                             .when(d.YEAR == 2013)
                             .then(d.POPU_2013_A)
                             .when(d.YEAR == 2014)
                             .then(d.POPU_2014_A)
                             .when(d.YEAR == 2015)
                             .then(d.POPU_2015_A)
                             .otherwise(None)
    )


    .with_columns(POPU_B = pl.when(d.YEAR == 2010)
                             .then(d.POPU_2010_B)
                             .when(d.YEAR == 2011)
                             .then(d.POPU_2011_B)
                             .when(d.YEAR == 2012)
                             .then(d.POPU_2012_B)
                             .when(d.YEAR == 2013)
                             .then(d.POPU_2013_B)
                             .when(d.YEAR == 2014)
                             .then(d.POPU_2014_B)
                             .when(d.YEAR == 2015)
                             .then(d.POPU_2015_B)
                             .otherwise(None)
    )

    .drop('POPU_2010_A', 'POPU_2010_B', 'POPU_2011_A', 'POPU_2011_B', 'POPU_2012_A', 'POPU_2012_B', 'POPU_2013_A', 'POPU_2013_B', 'POPU_2014_A', 'POPU_2014_B', 'POPU_2015_A', 'POPU_2015_B')

    .with_columns(EVO_POPU_A = pl.when(d.YEAR == 2010)
                                      .then(d.EVO_POPU_1011_A)
                                      .when(d.YEAR == 2011)
                                      .then(d.EVO_POPU_1112_A)
                                      .when(d.YEAR == 2012)
                                      .then(d.EVO_POPU_1213_A)
                                      .when(d.YEAR == 2013)
                                      .then(d.EVO_POPU_1314_A)
                                      .when(d.YEAR == 2014)
                                      .then(d.EVO_POPU_1415_A)
                                      .otherwise(None)
    )


    .with_columns(EVO_POPU_B = pl.when(d.YEAR == 2010)
                                      .then(d.EVO_POPU_1011_B)
                                      .when(d.YEAR == 2011)
                                      .then(d.EVO_POPU_1112_B)
                                      .when(d.YEAR == 2012)
                                      .then(d.EVO_POPU_1213_B)
                                      .when(d.YEAR == 2013)
                                      .then(d.EVO_POPU_1314_B)
                                      .when(d.YEAR == 2014)
                                      .then(d.EVO_POPU_1415_B)
                                      .otherwise(None)
    )

    .drop('EVO_POPU_1011_A', 'EVO_POPU_1112_A', 'EVO_POPU_1213_A', 'EVO_POPU_1314_A', 'EVO_POPU_1415_A', 'EVO_POPU_1011_B', 'EVO_POPU_1112_B', 'EVO_POPU_1213_B', 'EVO_POPU_1314_B', 'EVO_POPU_1415_B')
    
    
    .with_columns(INC_LC_A = pl.when(d.YEAR == 2010)
                               .then(d.INC_2010_LC_A)
                               .when(d.YEAR == 2011)
                               .then(d.INC_2011_LC_A)
                               .when(d.YEAR == 2012)
                               .then(d.INC_2012_LC_A)
                               .when(d.YEAR == 2013)
                               .then(d.INC_2013_LC_A)
                               .when(d.YEAR == 2014)
                               .then(d.INC_2014_LC_A)
                               .when(d.YEAR == 2015)
                               .then(d.INC_2015_LC_A)
                               .otherwise(None)
    )

    .with_columns(INC_LC_B = pl.when(d.YEAR == 2010)
                               .then(d.INC_2010_LC_B)
                               .when(d.YEAR == 2011)
                               .then(d.INC_2011_LC_B)
                               .when(d.YEAR == 2012)
                               .then(d.INC_2012_LC_B)
                               .when(d.YEAR == 2013)
                               .then(d.INC_2013_LC_B)
                               .when(d.YEAR == 2014)
                               .then(d.INC_2014_LC_B)
                               .when(d.YEAR == 2015)
                               .then(d.INC_2015_LC_B)
                               .otherwise(None)
    )


    .drop('INC_2010_LC_A','INC_2011_LC_A','INC_2012_LC_A','INC_2013_LC_A','INC_2014_LC_A','INC_2015_LC_A','INC_2010_LC_B','INC_2011_LC_B','INC_2012_LC_B','INC_2013_LC_B','INC_2014_LC_B','INC_2015_LC_B')


    .with_columns(EVO_INC_LC_A = pl.when(d.YEAR == 2010)
                                      .then(d.EVO_INC_LC_1011_A)
                                      .when(d.YEAR == 2011)
                                      .then(d.EVO_INC_LC_1112_A)
                                      .when(d.YEAR == 2012)
                                      .then(d.EVO_INC_LC_1213_A)
                                      .when(d.YEAR == 2013)
                                      .then(d.EVO_INC_LC_1314_A)
                                      .when(d.YEAR == 2014)
                                      .then(d.EVO_INC_LC_1415_A)
                                      .otherwise(None)
    )


    .with_columns(EVO_INC_LC_B = pl.when(d.YEAR == 2010)
                                      .then(d.EVO_INC_LC_1011_B)
                                      .when(d.YEAR == 2011)
                                      .then(d.EVO_INC_LC_1112_B)
                                      .when(d.YEAR == 2012)
                                      .then(d.EVO_INC_LC_1213_B)
                                      .when(d.YEAR == 2013)
                                      .then(d.EVO_INC_LC_1314_B)
                                      .when(d.YEAR == 2014)
                                      .then(d.EVO_INC_LC_1415_B)
                                      .otherwise(None)
    )

    .drop('EVO_INC_LC_1011_A', 'EVO_INC_LC_1112_A', 'EVO_INC_LC_1213_A', 'EVO_INC_LC_1314_A', 'EVO_INC_LC_1415_A', 'EVO_INC_LC_1011_B', 'EVO_INC_LC_1112_B', 'EVO_INC_LC_1213_B', 'EVO_INC_LC_1314_B', 'EVO_INC_LC_1415_B' )

    .with_columns(INC_USD2019_A = pl.when(d.YEAR == 2010)
                                    .then(d.INC_2010_USD19_A)
                                    .when(d.YEAR == 2011)
                                    .then(d.INC_2011_USD19_A)
                                    .when(d.YEAR == 2012)
                                    .then(d.INC_2012_USD19_A)
                                    .when(d.YEAR == 2013)
                                    .then(d.INC_2013_USD19_A)
                                    .when(d.YEAR == 2014)
                                    .then(d.INC_2014_USD19_A)
                                    .when(d.YEAR == 2015)
                                    .then(d.INC_2015_USD19_A)
                                    .otherwise(None)
    )

    .with_columns(INC_USD2019_B = pl.when(d.YEAR == 2010)
                                    .then(d.INC_2010_USD19_B)
                                    .when(d.YEAR == 2011)
                                    .then(d.INC_2011_USD19_B)
                                    .when(d.YEAR == 2012)
                                    .then(d.INC_2012_USD19_B)
                                    .when(d.YEAR == 2013)
                                    .then(d.INC_2013_USD19_B)
                                    .when(d.YEAR == 2014)
                                    .then(d.INC_2014_USD19_B)
                                    .when(d.YEAR == 2015)
                                    .then(d.INC_2015_USD19_B)
                                    .otherwise(None)
    )


    .drop('INC_2010_USD19_A', 'INC_2011_USD19_A', 'INC_2012_USD19_A', 'INC_2013_USD19_A', 'INC_2014_USD19_A', 'INC_2015_USD19_A', 'INC_2010_USD19_B', 'INC_2011_USD19_B', 'INC_2012_USD19_B', 'INC_2013_USD19_B', 'INC_2014_USD19_B', 'INC_2015_USD19_B')

    .with_columns(EVO_INC_USD2019_A = pl.when(d.YEAR == 2010)
                                    .then(d.EVO_INC_USD19_1011_A)
                                    .when(d.YEAR == 2011)
                                    .then(d.EVO_INC_USD19_1112_A)
                                    .when(d.YEAR == 2012)
                                    .then(d.EVO_INC_USD19_1213_A)
                                    .when(d.YEAR == 2013)
                                    .then(d.EVO_INC_USD19_1314_A)
                                    .when(d.YEAR == 2014)
                                    .then(d.EVO_INC_USD19_1415_A)
                                    .otherwise(None)
    )


    .with_columns(EVO_INC_USD2019_B = pl.when(d.YEAR == 2010)
                                    .then(d.EVO_INC_USD19_1011_B)
                                    .when(d.YEAR == 2011)
                                    .then(d.EVO_INC_USD19_1112_B)
                                    .when(d.YEAR == 2012)
                                    .then(d.EVO_INC_USD19_1213_B)
                                    .when(d.YEAR == 2013)
                                    .then(d.EVO_INC_USD19_1314_B)
                                    .when(d.YEAR == 2014)
                                    .then(d.EVO_INC_USD19_1415_B)
                                    .otherwise(None)
    )

    .drop('EVO_INC_USD19_1011_A', 'EVO_INC_USD19_1112_A', 'EVO_INC_USD19_1213_A', 'EVO_INC_USD19_1314_A', 'EVO_INC_USD19_1415_A', 'EVO_INC_USD19_1011_B', 'EVO_INC_USD19_1112_B', 'EVO_INC_USD19_1213_B', 'EVO_INC_USD19_1314_B', 'EVO_INC_USD19_1415_B')


    .with_columns(TAG_DURATION_OPENING = pl.when(d.IS_OPENING & (d.DURATION_FIRST_OPENING <= 3))
                                           .then(pl.lit('SHORT_OPENING'))
                                           .when(d.IS_OPENING & (d.DURATION_FIRST_OPENING > 3))
                                           .then(pl.lit('LONG_OPENING'))
                                           .otherwise(pl.lit('NO_OPENING'))
    )

    .with_columns(POPU_LOG_A = d.POPU_A.log(),
                  POPU_LOG_B = d.POPU_B.log(),
                  INC_LC_LOG_A = d.INC_LC_A.log(),
                  INC_LC_LOG_B = d.INC_LC_B.log(),
                  )


    .filter(d.YEAR != 2015) ## 4_308 to 3_525 ## for the evolution
    
    .join(df_opening_apt_pair, how = 'left', on = ['APT_CODE_A', 'APT_CODE_B'])

    .with_columns(TAG_NEW_OPENING = pl.when(d.IS_OPENING)
                                      .then(pl.lit('NEW_OPENING'))
                                      .when(d.NEW_OPENING)
                                      .then(pl.lit('HAS_OPENED'))
                                      .otherwise(pl.lit('OLD_2010'))
                 )


    .select(['YEAR',
             'IS_OPENING',
             'TAG_DURATION_OPENING',
             'TAG_NEW_OPENING',
             'IS_SPECIAL_A',
             'IS_CAPITAL_A',
             'IS_GLOBAL_HUB_A',
             'IS_DOMESTIC_HUB_A',
             'IS_SPECIAL_B',
             'IS_CAPITAL_B',
             'IS_GLOBAL_HUB_B',
             'IS_DOMESTIC_HUB_B',
             'POPU_A',
             'POPU_B',
             'EVO_POPU_A',
             'EVO_POPU_B',
             'INC_LC_A',
             'INC_LC_B',
             'EVO_INC_LC_A',
             'EVO_INC_LC_B',
             'INC_USD2019_A',
             'INC_USD2019_B',
             'EVO_INC_USD2019_A',
             'EVO_INC_USD2019_B',
             'POPU_LOG_A',
             'POPU_LOG_B',
             'INC_LC_LOG_A',
             'INC_LC_LOG_B',
            ])

)

# Save parquet

In [34]:
df_prep_for_stat.write_parquet("df_cities_metrics_preprocessed_for_stat.parquet")